In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
from tensorflow import keras

In [4]:
# tf.function and autograph
# 把python函数变成TensorFlow中的图
# autograph把python代码变成Tensorflow中的图

def scaled_elu(z,scale=1.0,alpha=1.0):
    # z>=0 ?  scale * z : scale*alpha*tf.nn.elu(z)
    is_positive = tf.greater_equal(z,0.0)
    return scale * tf.where(is_positive,z,alpha*tf.nn.elu(z))

print(scaled_elu(tf.constant(-3.)))
print(scaled_elu(tf.constant([-3,-2.5])))

scaled_elu_tf = tf.function(scaled_elu)
print(scaled_elu_tf(tf.constant(-3.)))
print(scaled_elu_tf(tf.constant([-3.,-2.5])))

print(scaled_elu_tf.python_function is scaled_elu)

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
True


In [5]:
%timeit scaled_elu(tf.random.normal((1000,1000)))
%timeit scaled_elu_tf(tf.random.normal((1000,1000)))

766 µs ± 25.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
768 µs ± 22.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
@tf.function(input_signature=[tf.TensorSpec([None],tf.int32,name='x')])
def cube(z):
    return tf.pow(z,3)

try:
    print(cube(tf.constant([1.,2.,3.])))
except ValueError as ex:
    print(ex)
    
print(cube(tf.constant([1,2,3])))


Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name='x'))
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [8]:
cube_func_int32 = cube.get_concrete_function(tf.TensorSpec([None],tf.int32)) #TensorSpec : 描述一个张量。
cube_func_int32

In [9]:
cube_func_int32.graph

In [10]:
cube_func_int32 is cube.get_concrete_function(tf.TensorSpec([5],tf.int32))

True

In [11]:
cube_func_int32 is cube.get_concrete_function(tf.constant([1,2,3]))

True

In [12]:
to_export =tf.Module()
to_export.cube = cube
tf.saved_model.save(to_export,"./signature_to_saved_model")

INFO:tensorflow:Assets written to: ./signature_to_saved_model/assets


In [13]:
!saved_model_cli show --dir ./signature_to_saved_model/ --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['x'] tensor_info:
        dtype: DT_INT32
        shape: (-1)
        name: serving_default_x:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_INT32
        shape: (-1)
        name: PartitionedCall:0
  Method name is: tensorflow/serving/predict


In [14]:
imported = tf.saved_model.load('./signature_to_saved_model')
imported.cube(tf.constant([2]))

<tf.Tensor: id=170568, shape=(1,), dtype=int32, numpy=array([8], dtype=int32)>